In [3]:
# Load packages
import numpy as np
import pandas as pd
import warnings, gc, sys, json
import datetime as dt
from m5_utils import *

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 100)

In [ ]:
import holidays

holidays_CA = []
holidays_TX = pd.DataFrame()
holidays_WI = pd.DataFrame()
holiday = []
description = []

for date, name in sorted(holidays.US(state=['CA'], years=[2011,2012,2013,2014,2015,2016]).items()):
    holiday.append(date)
    description.append(name)
holidays_CA = pd.DataFrame([holiday, description])
holidays_CA = holidays_CA.T
holidays_CA.columns = ['Holiday_Date','Holiday_Description']

for date, name in sorted(holidays.US(state=['TX'], years=[2011,2012,2013,2014,2015,2016]).items()):
    holiday.append(date)
    description.append(name)
holidays_TX = pd.DataFrame([holiday, description])
holidays_TX = holidays_TX.T
holidays_TX.columns = ['Holiday_Date','Holiday_Description']

for date, name in sorted(holidays.US(state=['WI'], years=[2011,2012,2013,2014,2015,2016]).items()):
    holiday.append(date)
    description.append(name)
holidays_WI = pd.DataFrame([holiday, description])
holidays_WI = holidays_WI.T
holidays_WI.columns = ['Holiday_Date','Holiday_Description']

holidays_CA.to_pickle('input/holidays_CA.pkl')
holidays_TX.to_pickle('input/holidays_TX.pkl')
holidays_WI.to_pickle('input/holidays_WI.pkl')


In [4]:
CA = pd.read_pickle('input/holidays_CA.pkl')
TX = pd.read_pickle('input/holidays_TX.pkl')
WI = pd.read_pickle('input/holidays_WI.pkl')

CA['state'] = 'CA'
TX['state'] = 'TX'
WI['state'] = 'WI'

weather['state_id'] = weather['state'].map(dict_m5_inv['state_id'])
del weather['state']
holidays_consolidated = pd.concat([CA, TX, WI], axis = 0); print(holidays_consolidated.shape)
holidays_consolidated.to_csv('input/holidays_consolidated.csv', index = False)

(396, 3)


In [ ]:
from wwo_hist import retrieve_hist_data

frequency = 6
start_date = '01-01-2011'
end_date = '31-12-2016'
api_key = '8cec0a01934b4ffe825150917201603'
location_list = ['california', 'texas', 'wisconsin']

In [ ]:
hist_weather_data = retrieve_hist_data(api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)

In [ ]:
def create_weather(filename):
    
    df = pd.read_csv(str(filename)+'.csv')
    to_drop = ['sunHour','maxtempC','mintempC','uvIndex','uvIndex.1','moon_illumination','moonrise','moonset','sunrise','sunset','DewPointC','WindGustKmph','cloudcover','pressure','visibility','winddirDegree','windspeedKmph']
    df.drop(to_drop, axis = 1, inplace = True)
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['year'] = df['date_time'].dt.year
    df['keep'] = df['hour'].apply(lambda x: 1 if (x>7 and x<23) else 0)
    df = df[df['keep']==1]
    del df['keep']
    df['date'] = (df.year.astype(str)+ '-' + df.month.astype(str)+'-' + df.day.astype(str))
    df = df.groupby(['date']).mean()
    droplist = ['hour','day','month','year']
    df.drop(droplist, inplace = True, axis = 1)
    for col in df.columns:
        if col!='date':
            df[col] = df[col].apply(lambda x: np.round(x,1))
    if filename=='california_1':
        new_col = 'CA'
    elif filename=='texas_1':
        new_col = 'TX'
    else:
        new_col = 'WI'
    df['state'] = new_col
    df = reduce_mem_usage(df)
    new_filename = 'new_' + filename + '.csv'
    df.to_csv(new_filename)
    
    return df

In [5]:
CA = create_weather('california_1')
TX = create_weather('texas_1')
WI = create_weather('wisconsin_1')

#CA = pd.read_csv('input/new_california_1.csv')
#TX = pd.read_csv('input/new_texas_1.csv')
#WI = pd.read_csv('input/new_wisconsin_1.csv')


weather_consolidated = pd.concat([CA, TX, WI], axis = 0); print(weather_consolidated.shape)
weather_consolidated.to_csv('input/weather_consolidated.csv', index = False)

(6576, 9)
